In [1]:
import glob
import numpy as np
import pandas as pd

In [2]:
#Helper functions
def readLocations(data):
    baseLongitudes=[]
    baseLatitudes=[]
    baseNames=[]
    for item in data:
        item=item.strip().split()
        baseLongitudes.append(float(item[1]))
        baseLatitudes.append(float(item[2]))
        baseNames.append(item[0])
    return baseLongitudes, baseLatitudes, baseNames
def processFile(fileExt, i):
    f=open('./stationsDetrendedBasicDownload/'+i+fileExt, "r")
    lat=f.readlines()
    lat2=[]
    for item in lat:
        item=item.strip()
        lat2.append(item)
    index=lat2.index("#       T            LDETRENDED       SIG_OBS")
    lat=lat2[index+1:]
    lat3=[]
    for item in lat:
        item=item.split()
        lat3.append(item)
    f.close()

    return lat3
def makeNumericMovingAverage(givenList):
    r=0
    for value in givenList:
        givenList[r]=float(value)
        r+=1
    numbers_series = pd.Series(givenList)
    indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=7)
    moving_averages_list=numbers_series.rolling(window=indexer, min_periods=1).mean()
    return moving_averages_list


In [3]:
lonFiles=glob.glob('./stationsDetrendedBasicDownload/*.lon')
latFiles=glob.glob('./stationsDetrendedBasicDownload/*.lat')
#Grab list of station names
names=[];
for i in latFiles:
    names.append(i.split("/")[2][0:4])

#Grab a list of station locations
f = open("sta_loc.txt", "r")
data=f.readlines()
[baseLongitudes, baseLatitudes, baseNames] = readLocations(data)
f.close()
f = open("stationLocations.txt", "r")
data=f.readlines()
[baseLongitudes2, baseLatitudes2, baseNames2] = readLocations(data)
f.close()
baseLongitudesTotal=baseLongitudes+baseLongitudes2
baseLatitudesTotal=baseLatitudes+baseLatitudes2
baseNamesTotal=baseNames+baseNames2

#Process each file
allLat=[]
allLon=[]
for i in names:
    lat=processFile('.lat', i)
    lon=processFile('.lon', i)

    try:
        indexName=baseNamesTotal.index(i)
    except: #station location is unknown
        continue
    baseLat=baseLatitudesTotal[indexName]
    baseLon=baseLongitudesTotal[indexName]
    allLat.append([i, baseLat, baseLon, [sublist[0] for sublist in lat],[sublist[1] for sublist in lat],[sublist[2] for sublist in lat] ])
    allLon.append([i, baseLat, baseLon, [sublist[0] for sublist in lon],[sublist[1] for sublist in lon],[sublist[2] for sublist in lon] ])

In [4]:
#Extract subset
latSimplified=[]
lonSimplified=[]
k=0
for i in allLat:
    baseLat=i[1]
    baseLon=i[2]
    if baseLat>42 and baseLat<52 and baseLon>-127 and baseLon<-115: #Pacific NW
        if baseLat>46.35 or baseLat<46.1 or baseLon>-122.1 or baseLon<-122.25: #ignore Mt. St. Helens
            latSimplified.append(i)
            lonSimplified.append(allLon[k])
    k+=1

In [6]:
#Convert to single timestream
f=open('./lonLat.txt', 'a')
f.close()
f=open('./names.txt', 'a')
f.close()
possibleValues=np.arange(1995.98588,2022.00218,0.00274)
x=0
for i in latSimplified:
    #Round to closest day
    newI=[]
    for j in i[3]:
        j=round(round(float(j)/0.00274, 0)*0.00274, 5)
        newI.append(j)
    latSimplified[x][3]=newI
    x+=1
x=0
for i in lonSimplified:
    newI=[]
    for j in i[3]:
        j=round(round(float(j)/0.00274, 0)*0.00274, 5)
        newI.append(j)
    lonSimplified[x][3]=newI
    
    #Make values numeric with a moving average
    latSimplified[x][4]=makeNumericMovingAverage(latSimplified[x][4])
    latSimplified[x][5]=makeNumericMovingAverage(latSimplified[x][5])
    lonSimplified[x][4]=makeNumericMovingAverage(lonSimplified[x][4])
    lonSimplified[x][5]=makeNumericMovingAverage(lonSimplified[x][5])

    #Ensure lat/lon in the same timestream
    ind_dict = dict((k,i) for i,k in enumerate(latSimplified[x][3]))
    inter = set(ind_dict).intersection(lonSimplified[x][3])
    indicesLat = sorted([ ind_dict[x] for x in inter ])
    ind_dict = dict((k,i) for i,k in enumerate(lonSimplified[x][3]))
    inter = set(ind_dict).intersection(latSimplified[x][3])
    indicesLon = sorted([ ind_dict[x] for x in inter ])
    latSimplified[x][3] = [latSimplified[x][3][i] for i in indicesLat]
    latSimplified[x][4] = [latSimplified[x][4][i] for i in indicesLat]
    latSimplified[x][5] = [latSimplified[x][5][i] for i in indicesLat]
    lonSimplified[x][3] = [lonSimplified[x][3][i] for i in indicesLon]
    lonSimplified[x][4] = [lonSimplified[x][4][i] for i in indicesLon]
    lonSimplified[x][5] = [lonSimplified[x][5][i] for i in indicesLon]
    
    
    #Write textfile
    data=[latSimplified[x][3], lonSimplified[x][4], latSimplified[x][4], lonSimplified[x][5], latSimplified[x][5]]
    name=latSimplified[x][0]
    with open('./stations/'+name+'.txt', 'w') as f:
        for time in np.arange(0,len(data[0])):
            line=[[i[time] for i in data]][0]
            line2=[]
            for thing in line:
                thing=str(thing)
                line2.append(str(thing))
            line=','.join(line2)
            f.write(line)
            f.write('\n')
    f.close()
    
    #Make lon/lat/name keys
    with open('./lonLat.txt', 'a') as f:
        f.write(",".join([str(latSimplified[x][2]), str(latSimplified[x][1])]))
        f.write('\n')
    f.close()
    with open('./names.txt', 'a') as f:
        f.write(str(latSimplified[x][0]))
        f.write('\n')
    f.close()

    x+=1    
